# Bilden eines Models mit Original und 2 Filtern

## Import der notwendigen Pakete

In [5]:
from sklearn import tree
import cv2 
import os
import glob
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

## Laden der notwendigen Daten

In [6]:
img_folder = "FACD_image/"
shape = (244,244)
random_state = 42

data_path = os.path.join(img_folder,'**')
files = glob.glob(data_path)

X = []
y = []

for f1 in files:
    data_path_folder = os.path.join(f1,'*g')
    files_folder = glob.glob(data_path_folder)
    for img_path in files_folder:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, shape)
        y.append(img_path.split("\\")[1])
        X.append(img.flatten())

## Datensätze vereinen und Model bilden

In [7]:
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier(n_estimators=20, random_state=random_state)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

## Crossvalidation

In [10]:
cv = RepeatedStratifiedKFold(n_splits=3, random_state=42)
n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
y_pred = classifier.predict(X_test)

from sklearn import metrics
print(metrics.classification_report(y_pred, y_test))

MemoryError: Unable to allocate 4.35 GiB for an array with shape (19626, 59536) and data type float32

## Ausgabe des Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',cmap="YlGnBu", linewidths=.5, cbar_kws={'orientation': 'vertical'})
plt.xlabel('true label')
plt.ylabel('predicted label');